<a href="https://colab.research.google.com/github/gladcolor/tweet_downloading_v2/blob/master/tweepy_v2_Ning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

The official [tweets searching progrom](https://github.com/twitterdev/search-tweets-python/tree/v2) is quite good, but I have test it yet. You can try it.


In [1]:
pip install searchtweets-v2

# Prerequisite functions

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import glob
import logging
import tweepy

In [6]:
def set_logger(log_file_path="debug.log", level="INFO"):
# def set_logger(log_file_path="debug.log", level="DEBUG"):
    logger = logging.getLogger()
    logger.setLevel(level)
    scream_handler = logging.StreamHandler()
    file_handler = logging.FileHandler(log_file_path)
    logger.addHandler(scream_handler)
    logger.addHandler(file_handler)
    return logger
    
  

try:
    # print(len(logger.handlers))
    while len(logger.handlers) > 1:
        logger.handlers.pop(0)
        # print(len(logger.handlers))
except:
    pass

logger = set_logger()

def get_api_token(token_path):    
    try:
        with open(token_path, "r") as f:
            logger.debug("token_path: %s" % token_path)
            lines = f.readlines()
            logger.debug("lines in the file: %s" % lines)

            lines = [line.split(": ")[-1][:-1] for line in lines]
        return lines

    except Exception as e:
        logger.error("Error: %s" % str(e))



# Set tokens

Put your Twitter API tokens in the ```tweet_api_keys.txt``` file in the same directory of this notebook in the following format:
```
Consumer API Key: XXXX
Consumer API Secret Key: XXXX
Bearer Token: XXXX
Access Token: XXXX
Access Token Secret: XXXX
```


In [7]:
token_path = r'/content/drive/MyDrive/Research/tweets_downloading/tweet_api_keys.txt'

tokens = get_api_token(token_path)

consumer_key = tokens[0]
consumer_secret = tokens[1]
bearer_token = tokens[2]
access_token = tokens[3]
access_token_secret = tokens[4]

# Download tweets

Please set ```query```, ```start_time```, ```end_time```, and ```max_results``` (10 - 500).

See these pages to building a query: 

[Building queries for Search Tweets](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#examples)

[Search Tweets](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all)

In [ ]:
import requests
import os
import json
import pandas as pd
import time

keyword = "vaccine"

# query = f"({keyword}) place_country:AU -is:retweet"
# query = f"({keyword}) place_country:AU"
query = f"({keyword})"
start_time = "2021-01-01T00:00:01Z"
end_time = "2021-06-01T00:00:01Z"
max_results = 500
# saved_path = os.path.join(os.getcwd(), "saved_tweets")
saved_path = os.path.join("/content/drive/MyDrive/Research/tweets_downloading/tweets_no_geo_vaccine")
# since_id = "139819805172285849"  # cannot used with start/end_time!


# borrow from Twitter:
# https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/master/Full-Archive-Search/full-archive-search.py


search_url = "https://api.twitter.com/2/tweets/search/all"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields

# a exmaple query to download tweets in Australia with a keyword of "vaccine" since 2020-01-01
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    # print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    # return response.json()
    return response.json()

def save_search(json_response, saved_path):
    try:
        if not os.path.exists(saved_path):
            os.mkdir(saved_path)

        meta = json_response['meta']
        data = json_response['data']
        includes = json_response['includes']
        basename = f"{meta['oldest_id']}_{meta['newest_id']}_{meta['result_count']}"

        data_filename = os.path.join(saved_path, basename + "_data.csv")
        df = pd.DataFrame(data)
        df.to_csv(data_filename, index=False)
        result_count = meta['result_count']
        result_count = str(result_count)
        logger.info("Saved %s tweets in: %s" % (result_count, data_filename))

        for key in includes.keys():
            includes_filename = os.path.join(saved_path, basename + f"_includes_{key}.csv")
            df = pd.DataFrame(includes[key])
            df.to_csv(includes_filename, index=False)
    except Exception as e:
        logger.error(e, exc_info=True)

def execute_download(saved_path=os.getcwd()):

    next_token = 'start'
    search_url = "https://api.twitter.com/2/tweets/search/all"
    headers = create_headers(bearer_token)
    total = 0
    query_params = {'query': query, \
                    "max_results": str(max_results), \
                    'expansions': 'attachments.poll_ids,attachments.media_keys,author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', \
                    'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld', \
                    'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type', \
                    "user.fields": 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',\
                    "media.fields": "duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics", \
                    "poll.fields": "duration_minutes,end_datetime,id,options,voting_status", \
                    "start_time": start_time, \
                    "end_time": end_time, \
                    # "since_id":since_id, \  # cannot used with start/end_time!
                    }

    while next_token != "":
        try:
            
            json_response = connect_to_endpoint(search_url, headers, query_params)
            df = pd.DataFrame(json_response['data'])
            save_search(json_response, saved_path)
            
            total += int(json_response['meta']['result_count'])
            logger.info("Downloaded %s tweets in total." % total)


            next_token = json_response['meta'].get('next_token', "")
            if next_token == "":
                print("No next page! Exit.")
                return

            query_params.update({"next_token": next_token})
            
            time.sleep(3)

        except Exception as e:
            logger.error(e, exc_info=True)
            time.sleep(3)
            continue


execute_download(saved_path=saved_path)